In [ ]:
import os
import pandas
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

# Define all variables and set up data import / export

### Data structure and setup:
It may be necessary to do multiple runs of this pipeline in order to compare results. This notebook presumes that the data is structured in this manner:

```drive/batch/run/images_corrected/barcoding/well-plate-site/results_files.csv```

You will define the first three variables below. Note that if one of these folders is missing (for example, you don't have separate run folders), then you can define that variable as an empty string, e.g. `run=''`, and it will be excluded from the path.

### Results output:
You can define the name of the folder you would like to contain your results in the `jupyter_output` variable below. The results folder is currently set up to be saved in a `run` folder within the `batch` folder; you can change that behavior by adjusting the `output_path` variable. Running the cell below will automatically create an output folder if it doesn't exist. 


In [ ]:
# set variables

# top level directory for the pooled cell painting analysis
drive = '/Users/pryder/GitHub/pearlryder_projects/2017_10_19_Profiling_rare_ORFs'

# folder for batch specific data and results
batch = '20210407_24W_CP251/'

run = '7_BC_Preprocess_Results/run3-FociTh1point5'

# folder w/ barcoding CSV result files
csvfolder = os.path.join(drive, batch, run, 'images_corrected/', 'barcoding/')

# define the name of a folder to save Jupyter notebook output
jupyter_output = 'Jupyter-output'

# create the output folder if it doesn't exist 
output_path = os.path.join(drive, batch, jupyter_output, run)

if not os.path.isdir(output_path):
    os.makedirs(output_path)
    

In [ ]:
# Useful function for merging CSVs contained in folders within one folder

def merge_csvs(csvfolder, filename, column_list=None):
    """ csvfolder is a path to a folder
    Iterates over all of the folders inside of that CSVfolder
    Merges the CSVs that match the filename into one dataframe 
    If a column list is passed, it keeps columns defined in the column list
    Prints a time stamp every 500 csvs
    Returns the merged dataframe
    """

    df_dict={}
    count = 0
    folderlist = os.listdir(csvfolder)
    print(count, datetime.datetime.ctime(datetime.datetime.now()))
    for eachfolder in folderlist:
            if os.path.isfile(os.path.join(csvfolder, eachfolder, filename)):
                if not column_list:
                    df_dict[eachfolder]=pandas.read_csv(os.path.join(csvfolder, eachfolder, filename),index_col=False)
                else:
                    df_dict[eachfolder]=pandas.read_csv(os.path.join(csvfolder, eachfolder, filename),index_col=False,usecols=column_list)
                count+=1
                if count % 500 == 0:
                    print(count, datetime.datetime.ctime(datetime.datetime.now()))
    print(count, datetime.datetime.ctime(datetime.datetime.now()))
    df_merged = pandas.concat(df_dict, ignore_index=True)
    print('done concatenating at', datetime.datetime.ctime(datetime.datetime.now()))
    
    return df_merged


# Merge CSVs

The first step is to merge the various CSVs for our experiment.
- The CSVs are stored in separate folders for each Well-Site and will be merged into one CSV in the next step.
- The `_BarcodeFoci.csv` files contain the % perfect match to the barcode score within the `Intensity_..._Barcodes_Scores` columns. 
- The `_Foci.csv` files contain the text for the barcode called and the barcode matched along with a matching score
- The Metadata columns are included in these CSVs 


In [ ]:
# read a single BarcodeFoci csv to determine the columns present in the data
filename = 'BarcodePreprocessing_Foci.csv'

sample_csv_df = pandas.read_csv(os.path.join(csvfolder, os.listdir(csvfolder)[0], filename))

# print the column list; copy-edit this list if desired 

print(list(sample_csv_df.columns))
# use this column list can be used to select the columns needed in the merged CSV, if desired  

In [ ]:
#Merge BarcodeFoci csvs
#Run if csvs are in separate folders
filename = 'BarcodePreprocessing_Foci.csv'
column_list = ['ImageNumber', 'ObjectNumber', 'Metadata_Plate', 'Metadata_Site', 'Metadata_Well', 'Metadata_Well_Value', 'Barcode_BarcodeCalled', 'Barcode_MatchedTo_Barcode', 'Barcode_MatchedTo_GeneCode', 'Barcode_MatchedTo_ID', 'Barcode_MatchedTo_Score']

df_foci = merge_csvs(csvfolder, filename, column_list)


In [ ]:
#Save the merged BarcodeFoci CSV
output_filename = 'BarcodePreprocessing_Foci_Merged.csv'

foci_path = os.path.join(output_path, output_filename)
df_foci.to_csv(foci_path)

# Load Merged CSVs

If you've already merged and save your CSVs using the code above, you can skip to this step of the notebook. 


In [ ]:
# List the contents of the Jupyter notebook output path
# See what CSVs have been saved and their names

drivelist = os.listdir(output_path)
filelist = [x for x in drivelist if not os.path.isdir(os.path.join(drive,x)) ]
print(filelist)

In [ ]:
# Load the CSVs into data frames

df_foci = pandas.read_csv(os.path.join(output_path, 'BarcodePreprocessing_Foci_Merged.csv'))
# df_cells=pandas.read_csv(os.path.join(drive, 'Cells_Merged.csv'))
# df_image=pandas.read_csv(os.path.join(drive, 'Image_Merged.csv'))

# Investigating barcode calling

### Distribution of barcode matching scores for all wells:

In [ ]:
# useful dataframe manipulations 

df_foci.sort_values(by=['Metadata_Well_Value', 'Metadata_Site'], inplace=True)

df_foci['well-site'] = df_foci['Metadata_Well_Value'] + '-' + df_foci['Metadata_Site'].astype(str)

df_foci_well_groups = df_foci.groupby("Metadata_Well_Value")

df_foci.head()

In [ ]:
sns.distplot(df_foci['Barcode_MatchedTo_Score'], kde=False)


distplot_filename = "barcode-score-distribution.png"
distplot_path = os.path.join(output_path, distplot_filename)

plt.savefig(distplot_path, dpi=300)

### Distribution of barcode matching scores by well:

In [ ]:
sns_displot = sns.displot(
    df_foci, x="Barcode_MatchedTo_Score", col="Metadata_Well_Value", col_wrap=4
)

displot_filename = "barcode-score-by-well.png"
displot_path = os.path.join(output_path, displot_filename)

sns_displot.savefig(displot_path, dpi=300)

### Barcodes called
The strings for the barcodes are saved in `Foci.csv` files for each well. Investigate: are the barcodes that are called most frequently all one letter? Are the bases (A, T, G, C) called an equal # of times? What percentage of barcodes are called at 100% accuracy? 

Since we have high variability between wells, we'll do this on a per well basis. That will help us to estimate if the barcodes are called inaccurately in different ways for each well. (For example, visual inspection of the Barcodes called for WellD4-Site41 suggests that "A" is being called at very high frequency).


In [ ]:
# Which barcodes are being called most often?
# Wells with barcodes like AAAAAAA called most often are probably low quality
# print to this notebook and save in a text file

barcode_filename = "barcodes-by-well.txt"
barcode_filepath = os.path.join(output_path, barcode_filename)

with open(barcode_filepath, "w") as barcode_file:

    for well, df_foci_well in df_foci_well_groups:
        print(well)
        print(df_foci_well['Barcode_BarcodeCalled'].value_counts().head(10), '\n')
        
        barcode_file.write(well + '\n')
        barcode_file.write(str(df_foci_well['Barcode_BarcodeCalled'].value_counts().head(10)) + '\n\n')

barcode_file.close()


In [ ]:
# What is the % of each base?

BarcodeCat = df_foci['Barcode_BarcodeCalled'].str.cat()

countG = BarcodeCat.count('G')
countT = BarcodeCat.count('T')
countA = BarcodeCat.count('A')
countC = BarcodeCat.count('C')

print ("Frequency of A is " + str(float(countA)/float((len(BarcodeCat)))*100))
print ("Frequency of C is " + str(float(countC)/float((len(BarcodeCat)))*100))
print ("Frequency of G is " + str(float(countG)/float((len(BarcodeCat)))*100))
print ("Frequency of T is " + str(float(countT)/float((len(BarcodeCat)))*100))

# 39% GC in the probes themselves; A & T should be roughly 30% each 


In [ ]:
# let's investigate % base distribution by well

well_site_ls = []
percent_close_ls = []
percent_perfect_ls = []
perfect_barcode_count_ls = []
frequency_A_ls = []
frequency_C_ls = []
frequency_G_ls = []
frequency_T_ls = []

for well, df_foci_well in df_foci_well_groups:
    BarcodeCat = df_foci_well['Barcode_BarcodeCalled'].str.cat()

    countG = BarcodeCat.count('G')
    countT = BarcodeCat.count('T')
    countA = BarcodeCat.count('A')
    countC = BarcodeCat.count('C')

    well_site_ls.append(df_foci_well['Metadata_Well_Value'].iloc[0])    
    percent_close_ls.append(sum(df_foci_well['Barcode_MatchedTo_Score']>=0.875)*100.0/sum(df_foci_well['Barcode_MatchedTo_Score']>0))
    percent_perfect_ls.append(sum(df_foci_well['Barcode_MatchedTo_Score']==1)*100.0/sum(df_foci_well['Barcode_MatchedTo_Score']>0))    
    perfect_barcode_count_ls.append(sum(df_foci_well['Barcode_MatchedTo_Score']==1))
    
    frequency_A_ls.append(float(countA)/float((len(BarcodeCat)))*100)
    frequency_C_ls.append(float(countC)/float((len(BarcodeCat)))*100)
    frequency_G_ls.append(float(countG)/float((len(BarcodeCat)))*100)
    frequency_T_ls.append(float(countT)/float((len(BarcodeCat)))*100)
        
percent_perfect_df = pandas.DataFrame({'Metadata_Well_Value': well_site_ls, 
                                       'percent-perfect': percent_perfect_ls,
                                       'percent-close': percent_close_ls, 
                                       'perfect-barcode-count': perfect_barcode_count_ls,
                                       'frequency-A': frequency_A_ls,
                                       'frequency-C': frequency_C_ls,
                                       'frequency-G': frequency_G_ls,
                                       'frequency-T': frequency_T_ls,
                                      })

In [ ]:
# save the % perfect dataframe to disk
percent_perfect_df_filename = 'percent-perfect.csv'

percent_perfect_path = os.path.join(output_path, percent_perfect_df_filename)

percent_perfect_df.to_csv(percent_perfect_path)

# % perfect, % close, and perfect barcode count data
percent_perfect_df